# PRE-PROCESSING et MACHINE LEARNING

## Importer les librairies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df=pd.read_csv("data_clean.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5246 entries, 0 to 16468
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    5246 non-null   object 
 1   Platform                5246 non-null   object 
 2   Year                    5246 non-null   int64  
 3   Genre                   5246 non-null   object 
 4   Publisher               5246 non-null   object 
 5   NA_Sales                5246 non-null   float64
 6   EU_Sales                5246 non-null   float64
 7   JP_Sales                5246 non-null   float64
 8   Other_Sales             5246 non-null   float64
 9   Global_Sales            5246 non-null   float64
 10  test_note_JVC           5246 non-null   float64
 11  avis_count_JVC          5246 non-null   float64
 12  avis_note_JVC           5246 non-null   float64
 13  Classification_Age_JVC  5246 non-null   float64
 14  Support_JVC             5246 non-null  